In [1]:
# Greenblatt's Magic Formula Implementation

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
tickers = ["AXP","AAPL","BA","CAT","CVX","CSCO","DIS","DOW", "XOM",
           "HD","IBM","INTC","JNJ","KO","MCD","MMM","MRK","MSFT",
           "NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT","WBA"]

In [4]:
#list of tickers whose financial data needs to be extracted
financial_dir = {}

for ticker in tickers:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c) "}) # try soup.findAll("table") if this line gives error 
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2])>0:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
        
        #combining all extracted information with the corresponding ticker
        financial_dir[ticker] = temp_dir
    except:
        print("Problem scraping data for ",ticker)

In [5]:
#storing information in pandas dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all NaN values
tickers = combined_financials.columns #updating the tickers list based on only those tickers whose values were successfully extracted
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]

In [7]:
combined_financials.head(50)

,AXP,AAPL,BA,CAT,CVX,CSCO,DIS,DOW,XOM,HD,...,MSFT,NKE,PFE,PG,TRV,UNH,VZ,V,WMT,WBA
Total assets,"19,83,21,000","32,38,88,000","13,36,25,000","7,84,53,000","23,74,28,000","9,48,53,000","20,15,49,000","6,05,24,000","36,25,97,000","5,12,36,000",...,"30,13,11,000","3,13,42,000","16,74,89,000","12,07,00,000","11,67,64,000","19,72,89,000","29,17,27,000","8,09,19,000","23,64,95,000","8,71,74,000"
Total liabilities,"17,52,50,000","25,85,49,000","14,19,25,000","6,38,24,000","9,22,20,000","5,69,33,000","11,32,86,000","4,64,30,000","16,36,59,000","5,43,52,000",...,"18,30,07,000","2,32,87,000","10,40,42,000","7,38,22,000","8,75,63,000","12,89,61,000","22,88,92,000","4,47,09,000","15,49,43,000","6,60,38,000"
Common stock,"1,63,000","5,07,79,000","50,61,000","59,35,000","18,32,000","4,12,02,000","5,44,97,000","8,000","1,56,37,000","89,000",...,"8,05,52,000","3,000","4,85,000","49,06,000","2,37,43,000",-,"4,29,000","50,86,000","2,84,000","12,000"
Retained earnings,"1,38,71,000","1,49,66,000","5,06,44,000","3,44,37,000","17,49,45,000","-27,63,000","3,83,15,000","1,70,45,000","42,13,41,000","5,17,29,000",...,"3,45,66,000","-1,91,000","9,76,70,000","10,02,39,000","3,87,71,000",-,"5,31,47,000","1,40,88,000","8,39,43,000","3,42,10,000"
Accumulated other comprehensive income,"-27,37,000","-4,06,000","-1,61,53,000","-15,67,000","-52,30,000","-5,19,000","-83,22,000","-1,02,46,000","-1,94,93,000","-7,39,000",...,"31,86,000","-56,000","-1,16,40,000","-1,72,45,000","25,02,000",-,"9,98,000","3,54,000","-1,28,05,000","-37,71,000"
Total stockholders' equity,"2,30,71,000","6,53,39,000","-86,17,000","1,45,88,000","14,42,13,000","3,79,20,000","8,35,83,000","1,35,41,000","19,16,50,000","-31,16,000",...,"11,83,04,000","80,55,000","6,31,43,000","4,65,21,000","2,92,01,000","6,83,28,000","6,13,95,000","3,62,10,000","7,46,69,000","2,06,37,000"
Total liabilities and stockholders' equity,"19,83,21,000","32,38,88,000","13,36,25,000","7,84,53,000","23,74,28,000","9,48,53,000","20,15,49,000","6,05,24,000","36,25,97,000","5,12,36,000",...,"30,13,11,000","3,13,42,000","16,74,89,000","12,07,00,000","11,67,64,000","19,72,89,000","29,17,27,000","8,09,19,000","23,64,95,000","8,71,74,000"
Total revenue,"2,81,60,000","27,38,57,000","6,07,65,000","4,36,57,000","10,42,02,000","4,80,71,000","7,51,25,000","3,80,40,000","19,58,60,000","12,56,31,000",...,"14,71,14,000","3,82,54,000","4,86,49,000","7,39,75,000","3,19,81,000","25,56,39,000","12,83,75,000","2,38,85,000","54,87,43,000","14,15,05,000"
Interest expense,"25,09,000","30,49,000","17,00,000","4,96,000","6,76,000","5,19,000","14,45,000","8,39,000","10,46,000","13,19,000",...,"25,43,000",NaN,"15,94,000","5,36,000","3,39,000","16,63,000","43,26,000","5,16,000","24,43,000","6,13,000"
Income before tax,"44,24,000","6,83,17,000","-81,93,000","44,19,000","-1,49,01,000","1,29,65,000","1,31,45,000","-19,41,000","41,44,000","1,63,33,000",...,"5,64,74,000","32,48,000","85,06,000","1,71,33,000","32,37,000","2,07,42,000","2,14,26,000","1,53,80,000","2,68,30,000","-6,59,000"


In [8]:
# Check scrapping code one more time. Take care that all numbers on yahoo finance are in multiple of thousand

# I think we need to change all the codes in below cells according to new yahoo finance web page. Also, we can extract this data from IEX cloud API. 

In [9]:
# creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & amortisation",
         "Market cap (intra-day)",
         "Net income available to common shareholders",
         "Net cash provided by operating activities",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Net property, plant and equipment",
         "Total stockholders' equity",
         "Long-term debt",
         "Forward annual dividend yield"] # change as required

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]
all_stats = {}
for ticker in tickers:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)


can't read data for  AXP
can't read data for  AAPL
can't read data for  BA
can't read data for  CAT
can't read data for  CVX
can't read data for  CSCO
can't read data for  DIS
can't read data for  DOW
can't read data for  XOM
can't read data for  HD
can't read data for  IBM
can't read data for  INTC
can't read data for  JNJ
can't read data for  KO
can't read data for  MCD
can't read data for  MMM
can't read data for  MRK
can't read data for  MSFT
can't read data for  NKE
can't read data for  PFE
can't read data for  PG
can't read data for  TRV
can't read data for  UNH
can't read data for  VZ
can't read data for  V
can't read data for  WMT
can't read data for  WBA


In [ ]:
# cleansing of fundamental data imported in dataframe
all_stats_df = pd.DataFrame(all_stats,index=indx)
all_stats_df[tickers] = all_stats_df[tickers].replace({',': ''}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'M': 'E+03'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'B': 'E+06'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'T': 'E+09'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'%': 'E-02'}, regex=True)
for ticker in all_stats_df.columns:
    all_stats_df[ticker] = pd.to_numeric(all_stats_df[ticker].values,errors='coerce')
all_stats_df.dropna(axis=1,inplace=True)
tickers = all_stats_df.columns

# calculating relevant financial metrics for each stock
transpose_df = all_stats_df.transpose()
final_stats_df = pd.DataFrame()
final_stats_df["EBIT"] = transpose_df["EBITDA"] - transpose_df["D&A"]
final_stats_df["TEV"] =  transpose_df["MarketCap"].fillna(0) \
                         +transpose_df["TotDebt"].fillna(0) \
                         -(transpose_df["CurrAsset"].fillna(0)-transpose_df["CurrLiab"].fillna(0))
final_stats_df["EarningYield"] =  final_stats_df["EBIT"]/final_stats_df["TEV"]
final_stats_df["FCFYield"] = (transpose_df["CashFlowOps"]-transpose_df["Capex"])/transpose_df["MarketCap"]
final_stats_df["ROC"]  = (transpose_df["EBITDA"] - transpose_df["D&A"])/(transpose_df["PPE"]+transpose_df["CurrAsset"]-transpose_df["CurrLiab"])
final_stats_df["BookToMkt"] = transpose_df["BookValue"]/transpose_df["MarketCap"]
final_stats_df["DivYield"] = transpose_df["DivYield"]


In [ ]:
################################Output Dataframes##############################

# finding value stocks based on Magic Formula
final_stats_val_df = final_stats_df.loc[tickers,:]
final_stats_val_df["CombRank"] = final_stats_val_df["EarningYield"].rank(ascending=False,na_option='bottom')+final_stats_val_df["ROC"].rank(ascending=False,na_option='bottom')
final_stats_val_df["MagicFormulaRank"] = final_stats_val_df["CombRank"].rank(method='first')
value_stocks = final_stats_val_df.sort_values("MagicFormulaRank").iloc[:,[2,4,8]]
print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")
print(value_stocks)


# finding highest dividend yield stocks
high_dividend_stocks = final_stats_df.sort_values("DivYield",ascending=False).iloc[:,6]
print("------------------------------------------------")
print("Highest dividend paying stocks")
print(high_dividend_stocks)


# # Magic Formula & Dividend yield combined
final_stats_df["CombRank"] = final_stats_df["EarningYield"].rank(ascending=False,method='first') \
                              +final_stats_df["ROC"].rank(ascending=False,method='first')  \
                              +final_stats_df["DivYield"].rank(ascending=False,method='first')
final_stats_df["CombinedRank"] = final_stats_df["CombRank"].rank(method='first')
value_high_div_stocks = final_stats_df.sort_values("CombinedRank").iloc[:,[2,4,6,8]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)
